In [1]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import glob
#import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
cudnn.benchmark = True
import time
import copy

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128 , 128)),   
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128 , 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'E:\datasets\augment'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
              for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) 
              for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=14):
    since = time.time()
    #Сохраняем  лучшие веса 
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 99
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)


        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            # перенос  вычислений на куду 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                
                optimizer.zero_grad()

                #
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    acc = (torch.sum(preds == labels.data)) / len(inputs)
                   

                    # считаем градиенты тольок если трейн данные 
                    if phase == 'train':
#                         loss.backward()
                         optimizer.step()

                # стасистика  
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # сохраняем не только лучшие веса , но и акураси 
#             if phase == 'val' and epoch_loss < best_loss:
#                 best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
#             if phase == 'val' and epoch_acc > best_acc:
#                 best_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_loss:4f}')
    

    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_conv = models.efficientnet_b4()
num_ftrs = model_conv.classifier[1].in_features
model_conv.fc = nn.Linear(num_ftrs, 11)
model_conv = model_conv.to(device)
for param in model_conv.parameters():
    param.requires_grad = False
model_conv.load_state_dict(torch.load('weights-no_weather_class-efinetb4-bestloss.pth')) #,map_location ='cpu'
criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.Adam(model_conv.classifier[1].parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [6]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler,
                       num_epochs=14)

Epoch 0/13
----------
train Loss: 0.4862 Acc: 0.8677
val Loss: 0.3961 Acc: 0.8934

Epoch 1/13
----------
train Loss: 0.4946 Acc: 0.8703
val Loss: 0.4243 Acc: 0.8890

Epoch 2/13
----------
train Loss: 0.4858 Acc: 0.8697
val Loss: 0.3958 Acc: 0.8925

Epoch 3/13
----------
train Loss: 0.4873 Acc: 0.8690
val Loss: 0.4080 Acc: 0.8906

Epoch 4/13
----------
train Loss: 0.4768 Acc: 0.8711
val Loss: 0.4036 Acc: 0.8936

Epoch 5/13
----------
train Loss: 0.4844 Acc: 0.8704
val Loss: 0.4137 Acc: 0.8902

Epoch 6/13
----------
train Loss: 0.4859 Acc: 0.8685
val Loss: 0.3949 Acc: 0.8957

Epoch 7/13
----------
train Loss: 0.4881 Acc: 0.8685
val Loss: 0.4207 Acc: 0.8848

Epoch 8/13
----------
train Loss: 0.4835 Acc: 0.8693
val Loss: 0.3961 Acc: 0.8936

Epoch 9/13
----------
train Loss: 0.5049 Acc: 0.8671
val Loss: 0.4108 Acc: 0.8890

Epoch 10/13
----------
train Loss: 0.4955 Acc: 0.8687
val Loss: 0.4169 Acc: 0.8860

Epoch 11/13
----------
train Loss: 0.4966 Acc: 0.8645
val Loss: 0.3999 Acc: 0.8936

Ep

In [7]:
nb_classes = 11
device = torch.device("cpu")
predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_conv(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
        from sklearn.metrics import classification_report
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния', 'Нет погоды' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [9]:
torch.save(model_conv.state_dict(), "weights-no_weather_class-efinetb4_conv_optimiser-bestloss.pth")

In [10]:
model_conv = models.efficientnet_b4()
num_ftrs = model_conv.classifier[1].in_features
model_conv.fc = nn.Linear(num_ftrs, 11)
model_conv = model_conv.to(device)
model_conv.load_state_dict(torch.load('weights-no_weather_class-efinetb4_conv_optimiser-bestloss.pth')) #,map_location ='cpu'


<All keys matched successfully>

In [ ]:
data_transforms = {
    'test': transforms.Compose([
        transforms.Resize((128 , 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'E:\datasets\split'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ["test"]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ["test"]}
dataset_sizes = {x: len(image_datasets[x]) for x in ["test"]}
class_names = image_datasets["test"].classes
device = torch.device("cpu")
nb_classes = 11

predlist=torch.zeros(0,dtype=torch.long, device=device)
lbllist=torch.zeros(0,dtype=torch.long, device=device)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_conv(inputs)
        _, preds = torch.max(outputs, 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])
        from sklearn.metrics import classification_report
target_names = ['ясно' ,'облачно' ,'туман' ,'мороз' ,'град' ,'молния', 'Нет погоды' , 'дождь', 'радуга',
           'снег', 'восход']
print(classification_report(lbllist.numpy(), predlist.numpy(),target_names=target_names))